# Unsupervised Lab Session

## Learning outcomes:
- Exploratory data analysis and data preparation for model building.
- PCA for dimensionality reduction.
- K-means and Agglomerative Clustering

## Problem Statement
Based on the given marketing campigan dataset, segment the similar customers into suitable clusters. Analyze the clusters and provide your insights to help the organization promote their business.

## Context:
- Customer Personality Analysis is a detailed analysis of a company’s ideal customers. It helps a business to better understand its customers and makes it easier for them to modify products according to the specific needs, behaviors and concerns of different types of customers.
- Customer personality analysis helps a business to modify its product based on its target customers from different types of customer segments. For example, instead of spending money to market a new product to every customer in the company’s database, a company can analyze which customer segment is most likely to buy the product and then market the product only on that particular segment.

## About dataset
- Source: https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis?datasetId=1546318&sortBy=voteCount

### Attribute Information:
- ID: Customer's unique identifier
- Year_Birth: Customer's birth year
- Education: Customer's education level
- Marital_Status: Customer's marital status
- Income: Customer's yearly household income
- Kidhome: Number of children in customer's household
- Teenhome: Number of teenagers in customer's household
- Dt_Customer: Date of customer's enrollment with the company
- Recency: Number of days since customer's last purchase
- Complain: 1 if the customer complained in the last 2 years, 0 otherwise
- MntWines: Amount spent on wine in last 2 years
- MntFruits: Amount spent on fruits in last 2 years
- MntMeatProducts: Amount spent on meat in last 2 years
- MntFishProducts: Amount spent on fish in last 2 years
- MntSweetProducts: Amount spent on sweets in last 2 years
- MntGoldProds: Amount spent on gold in last 2 years
- NumDealsPurchases: Number of purchases made with a discount
- AcceptedCmp1: 1 if customer accepted the offer in the 1st campaign, 0 otherwise
- AcceptedCmp2: 1 if customer accepted the offer in the 2nd campaign, 0 otherwise
- AcceptedCmp3: 1 if customer accepted the offer in the 3rd campaign, 0 otherwise
- AcceptedCmp4: 1 if customer accepted the offer in the 4th campaign, 0 otherwise
- AcceptedCmp5: 1 if customer accepted the offer in the 5th campaign, 0 otherwise
- Response: 1 if customer accepted the offer in the last campaign, 0 otherwise
- NumWebPurchases: Number of purchases made through the company’s website
- NumCatalogPurchases: Number of purchases made using a catalogue
- NumStorePurchases: Number of purchases made directly in stores
- NumWebVisitsMonth: Number of visits to company’s website in the last month

### 1. Import required libraries

In [ ]:
import numpy as np
from numpy.linalg import eig
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
import warnings
warnings.filterwarnings('ignore')

### 2. Load the CSV file (i.e marketing.csv) and display the first 5 rows of the dataframe. Check the shape and info of the dataset.

In [ ]:
df = pd.read_csv('marketing.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

### 3. Check the percentage of missing values? If there is presence of missing values, treat them accordingly.

In [ ]:
df.isnull().sum()/len(df)*100

In [ ]:
df['Income'] = df['Income'].fillna(df['Income'].median())

In [ ]:
df.isnull().sum()/len(df)*100

### 4. Check if there are any duplicate records in the dataset? If any drop them.

In [ ]:
len(df[df.duplicated()])

### 5. Drop the columns which you think redundant for the analysis 

In [ ]:
df = df.drop(columns=['ID','Dt_Customer'],axis=1)

### 6. Check the unique categories in the column 'Marital_Status'
- i) Group categories 'Married', 'Together' as 'relationship'
- ii) Group categories 'Divorced', 'Widow', 'Alone', 'YOLO', and 'Absurd' as 'Single'.

In [ ]:
df['Marital_Status'].unique()

In [ ]:
df['Marital_Status'] = df['Marital_Status'].replace(['Married','Together'],'relationship')

In [ ]:
df['Marital_Status'] = df['Marital_Status'].replace(['Single','Divorced','Widow','Alone','Absurd','YOLO'],'Single')

In [ ]:
df['Marital_Status'].unique()

### 7. Group the columns 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', and 'MntGoldProds' as 'Total_Expenses'

In [ ]:
df['Total_Expenses']  = df['MntWines'] + df['MntFruits'] + df['MntMeatProducts'] + df['MntFishProducts'] + df['MntSweetProducts'] + df['MntGoldProds']

In [ ]:
df.columns

### 8. Group the columns 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', and 'NumDealsPurchases' as 'Num_Total_Purchases'

In [ ]:
df['Num_Total_Purchases']  = df['NumWebPurchases'] + df['NumCatalogPurchases'] + df['NumStorePurchases'] + df['NumDealsPurchases']

In [ ]:
df.columns

### 9. Group the columns 'Kidhome' and 'Teenhome' as 'Kids'

In [ ]:
df['Kids']  = df['Kidhome'] + df['Teenhome'] 

In [ ]:
df.columns

In [ ]:
df['Kids']

### 10. Group columns 'AcceptedCmp1 , 2 , 3 , 4, 5' and 'Response' as 'TotalAcceptedCmp'

In [ ]:
df['TotalAcceptedCmp'] = df['AcceptedCmp1'] + df['AcceptedCmp2'] + df['AcceptedCmp3']+ df['AcceptedCmp4'] + df['AcceptedCmp5'] + df['Response']

In [ ]:
df.columns

In [ ]:
df['TotalAcceptedCmp']

### 11. Drop those columns which we have used above for obtaining new features

In [ ]:
df = df.drop(columns=['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts','MntGoldProds', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases','NumDealsPurchases','Kidhome','Teenhome','AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','Response'])

### 12. Extract 'age' using the column 'Year_Birth' and then drop the column 'Year_birth'

In [ ]:
df['Age'] = 2023 - df['Year_Birth']

In [ ]:
df = df.drop(columns=['Year_Birth'])

### 13. Encode the categorical variables in the dataset

In [ ]:
#Label encoding
cate=['Education','Marital_Status']
lbl_encode = LabelEncoder()
for i in cate:
    df[i]=df[[i]].apply(lbl_encode.fit_transform)

In [ ]:
cate

### 14. Standardize the columns, so that values are in a particular range

In [ ]:
#Standaedization
df1 = df.copy()
scaled_features = StandardScaler().fit_transform(df1.values)
scaled_features_df = pd.DataFrame(scaled_features,index=df1.index, columns=df1.columns)

In [ ]:
scaled_features_df.head(3)

### 15. Apply PCA on the above dataset and determine the number of PCA components to be used so that 90-95% of the variance in data is explained by the same.

In [ ]:
#step 1:calculating covariance matrix
cov_matrix = np.cov(scaled_features.T)
cov_matrix

In [ ]:
#step2: calculating the eigen values,eigen vectors
eigenvalues, eigenvectors = eig(cov_matrix)
print('eigein vals:','\n',eigenvalues)
print('\n')
print('eigein vectors:','\n',eigenvectors)

In [ ]:
#step3: scree plot
total = sum(eigenvalues)
var_exp = [(i/total)*100 for i in sorted(eigenvalues,reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print('Explained variance:',var_exp)
print('\n')
print('Cummulative variance Explained :',cum_var_exp)


In [ ]:
#scree plot
plt.bar(range(11),var_exp,align='center',color='lightgreen',edgecolor='black',label='Explained variance')
plt.step(range(11),cum_var_exp,where='mid',color='red',label='Cummulative variance Explained')
plt.xlabel('Principal Components')
plt.ylabel('Explained variance ratio')
plt.title('Scree plot')
plt.legend(loc='best')
plt.show()

#.we can observe from the above scree plot the first 8 principal components are explaining about 90-95% of the variation,so we can choose opotimal no. of principal component as 8.

### 16. Apply K-means clustering and segment the data (Use PCA transformed data for clustering)

In [ ]:
#Using PCA transformed data for clustering
pca = PCA(n_components=8)

pca_df = pd.DataFrame(pca.fit_transform(scaled_features_df),columns=['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8'])
pca_df.head()

In [ ]:
#finding optimal k value by kmeans clustering using Elbow plot
cluster_errors = []
cluster_range = range(2,15)
for num_clusters in cluster_range:
    clusters = KMeans(num_clusters,random_state=100)
    clusters.fit(pca_df)
    cluster_errors.append(clusters.inertia_)

In [ ]:
#creating dataframe of np. of clusters and clusters error
cluster_df = pd.DataFrame({'num_clusters':cluster_range,'cluster_errors':cluster_errors})

##Elbow Plot
plt.figure(figsize=[15,5])
plt.plot(cluster_df['num_clusters'],cluster_df['cluster_errors'],marker='o',color='b')
plt.show()

In [ ]:
##applyning kmeans clustering for optimal no.of clusters abtained above
kmeans = KMeans(n_clusters=3,random_state=100)
kmeans.fit(pca_df)

In [ ]:
##creating dataframes of the label
label = pd.DataFrame(kmeans.labels_,columns=['Label'])


In [ ]:
##joining label dataframe tp pca_df dataframes
kmeans_df = pca_df.join(label)
kmeans_df.head()

In [ ]:
kmeans_df['Label'].value_counts()

In [ ]:
##visualizating the clusters formed
sns.scatterplot(kmeans_df['PC1'],kmeans_df['PC2'],hue='Label',data = kmeans_df)
plt.show()

### 17. Apply Agglomerative clustering and segment the data (Use Original data for clustering), and perform cluster analysis by doing bivariate analysis between the cluster label and different features and write your observations.

In [ ]:
#Agglomerative clustering using original data
#from scipy.cluster.hierarchy import dendogram, linkage

plt.figure(figsize=[18,5])
merg = linkage(scaled_features,method='ward')
dendrogram (merg,leaf_rotation=90,)
plt.xlabel('Datapoints')
plt.ylabel('Euclidean distance')
plt.show()

In [ ]:
#computing silhouette score for Agglomerative clustering
from sklearn.metrics import silhouette_score

In [ ]:
for i in range(2,15):
    hier = AgglomerativeClustering(n_clusters=i)
    hier = hier.fit(scaled_features_df)
    labels = hier.fit_predict(scaled_features_df)
    print(i,silhouette_score(scaled_features_df,labels))

*we can observe from the above dendograms and silhoutte score for diffrent clusters and score is highest for cluster3.
*lets choose K=3 as the optimal cluster size and build the agglomerative clustering modelusing original dataset.

In [ ]:
#build the agglomerative clustering modelusing original dataset.
hie_cluster = AgglomerativeClustering(n_clusters=3,affinity='euclidean',linkage='ward')
hie_cluster_model = hie_cluster.fit(scaled_features_df)

In [ ]:
#creating dataframe of labels
df_label1 = pd.DataFrame(hie_cluster_model.labels_,columns=['Labels'])
df_label1.head(5)

In [ ]:
#joining the label dataframe with the unscaled initial cleaned dataframe(dfc)

df_hier = df.join(df_label1)
df_hier.head()

### Visualization and Interpretation of results

In [ ]:
sns.barplot (df_hier['Labels'], df_hier['Total_Expenses'])
plt.show()

In [ ]:
sns.barplot(df_hier['Labels'],df_hier['Income'])
plt.show()

#.Total no.of purchases is much higher from cluster 0 compared to cluster 1 and 2.
#.So we can conclude that total no.of purchases is much higher from cluster 0 ,has spent maximum ammount and has maximum income
#.Customer from cluster 1 has least Total_expenses,minimum balance and hence do minimum purchse compared to other two clusters.
#.Customers from cluster 2 does average purchase and has average income.

-----
## Happy Learning
-----